In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
plt.rc('xtick',c='w')
plt.rc('ytick',c='w')
from PyAstronomy import pyasl
from astropy.time import Time
from astropy import units as u
from astropy.modeling import Fittable1DModel, Parameter
from astropy.modeling import models, fitting
from lmfit import Parameters, minimize, report_fit, Model,fit_report
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.coordinates import ICRS
from pylab import rc
from astropy.timeseries import LombScargle

/home/nwal/anaconda3/lib/python3.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
def Read_Three_Column_File(file_name):
    with open(file_name, 'r') as data:
        x = []
        y = []
        shift = []
        for line in data:
            p = line.split()
            shift.append(float(p[0]))
            x.append(float(p[1]))
            y.append(float(p[2]))

    return x, y,shift


In [16]:
#isaac
#x28Ph, y28Ph,_= Read_Three_Column_File('/home/nwal/Desktop/GD1400/jayCor/28/shiftedAir.txt')
x28Ph, y28Ph,_= Read_Three_Column_File('/home/nwal/Desktop/GD1400/jayCor/28/fxcorPhoenixPy.txt')
path = '/home/nwal/Desktop/GD1400/oct28/all/proc2/'
airmasses = []
with open(path+'ToCont') as f:
    content = f.readlines()
content = [x.strip() for x in content] 
times = []
dateObs28 = []
for el in content:
    fits_image_filename = path+el
    hdul = fits.open(fits_image_filename)
    hdr = hdul[0].header
    timeHeader = hdr.comments['UTC'][:10]
    expTime = hdr['EXPTIME']
    date = hdr['DATE-OBS']
    dateObs28.append(date)
    airmass = hdr['HIERARCH ESO TEL AIRM START']
    airmasses.append(airmass)
    t = Time(date, format='isot', scale='utc')
    t = t + (expTime/2*u.second)
    times.append(t.jd)
times28 = times
dateObs29 = []
#x29Ph, y29Ph,_= Read_Three_Column_File('/home/nwal/Desktop/GD1400/jayCor/29/shiftedAir.txt')
x29Ph, y29Ph,_= Read_Three_Column_File('/home/nwal/Desktop/GD1400/jayCor/29/fxcorPhoenixPy.txt')
path = '/home/nwal/Desktop/GD1400/oct29/all/forProc2/'
with open(path+'CleanFrames') as f:
    content = f.readlines()
content = [x.strip() for x in content] 
times = []
airmasses = []
for el in content:
    fits_image_filename = path+el
    hdul = fits.open(fits_image_filename)
    hdr = hdul[0].header
    timeHeader = hdr.comments['UTC'][:10]
    expTime = hdr['EXPTIME']
    date = hdr['DATE-OBS']
    dateObs29.append(date)
    t = Time(date, format='isot', scale='utc')
    airmass = hdr['HIERARCH ESO TEL AIRM START']
    airmasses.append(airmass)
    t = t + (expTime/2*u.second)
    times.append(t.jd)
times29 = times

In [12]:
times28[0]

2455498.6071224944

In [17]:
path = '/home/nwal/Desktop/GD1400/jayCor/28/'
with open(path+'cont') as f:
    content = f.readlines()
content = [x.strip() for x in content]
inputNames = content
with open(path+'headers') as f:
    content = f.readlines()
content = [x.strip() for x in content]
outputNames = content
count = 0
for el in inputNames:
    fits_image_filename = path+el
    with fits.open(fits_image_filename) as hdul:
        hdr = hdul[0].header
        hdr.set('OBSERVAT', 'esovlt')
        hdr.set('RA', '01:47:21.8246239967')
        hdr.set('DEC', '-21:56:51.387203270')
        hdr.set('UTMIDDLE', times28[count])
        hdr.set('EXPTIME', '600')
        hdr.set('EPOCH', '2000')
        hdr.set('DATE-OBS', dateObs28[count])
        outName = path + outputNames[count]
        count = count +1
        hdul.writeto(outName)

with fits.open('/home/nwal/Desktop/GD1400/jayCor/28/PhoenixResampledG.fits') as hdul:
    hdr = hdul[0].header
    hdr.set('OBSERVAT', 'kpno')
    hdr.set('RA', '12:00:00')#'26.8412500')
    hdr.set('DEC', '12:00:00')#'-21.9494167')
    #hdr.set('UT', '2455498.6071224944')
    #hdr.set('UTMIDDLE', '2455498.6101224944')
    hdr.set('EXPTIME', '0')
    hdr.set('EPOCH', '1993.0')
    hdr.set('DATE-OBS', '1993-03-17T04:56:38.0')
    hdul.writeto('/home/nwal/Desktop/GD1400/jayCor/28/phoenixHeader.fits')